In [1]:
%load_ext autoreload

In [2]:
%autoreload
import sys
import copy
sys.path.append('../../style_generation_pipeline/')

import generate_explanations
from  styles import StyleGenerator

In [3]:
import json
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import explanation_interfaces
import random
import tabulate
import glob

pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
model_path = 'aa_model-luar'
input_path = '/mnt/swordfish-pool2/milad/hiatus-data/explainability-pilot-samples/'
#interp_space_path = '/mnt/swordfish-pool2/milad/hiatus-data/explainability_all_data/interpretable_space.pkl'
#interp_space_rep_path = '/mnt/swordfish-p'ool2/milad/hiatus-data/explainability_all_data/interpretable_space_representations.json'
interp_space_path = '/mnt/swordfish-pool2/milad/hiatus-data/explainability_all_data/interp_space_126_clusters/interpretable_space.pkl'
interp_space_rep_path = '/mnt/swordfish-pool2/milad/hiatus-data/explainability_all_data/interp_space_126_clusters/interpretable_space_representations.json'

In [5]:
style_generator = StyleGenerator(model_name="openai:gpt-3.5-turbo", device=0, max_new_tokens=500)
#style_generator = StyleGenerator(model_name="openai:gpt-4", device=0, max_new_tokens=500)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /mnt/swordfish-pool2/milad/hf-cache-new/token
Login successful


In [6]:
def prepare_data_for_evaluation1(input_path, interp_space_path, interp_space_rep_path, feat_clm='llm_tfidf_rep', top_c=1, top_k=5):
    # Load all documents
    candidate_documents_df = pd.concat([pd.read_json(p, lines=True) for p in glob.glob(input_path + '/*/data/*_candidates.jsonl')])
    query_documents_df = pd.concat([pd.read_json(p, lines=True) for p in glob.glob(input_path + '/*/data/*_queries.jsonl')])
    query_documents_df['authorID'] = query_documents_df.authorIDs.apply(lambda x: x[0])
    candidate_documents_df['authorID'] = candidate_documents_df.authorSetIDs.apply(lambda x: x[0])
    all_documents_df = pd.concat([query_documents_df, candidate_documents_df])

    # Get the style features
    document_rep_feats, _ = generate_explanations.get_documents_style_descriptions(all_documents_df.fullText.tolist(), model_path, interp_space_path, interp_space_rep_path, feat_clm, top_c=top_c, top_k=top_k)
    document_distracting_feats, _ = generate_explanations.get_documents_style_descriptions(all_documents_df.fullText.tolist(), model_path, interp_space_path, interp_space_rep_path, feat_clm, top_c=top_c, top_k=top_k, flip_cluster_order=True)

    
    # Add features to the dataframe
    all_documents_df['describtive_feats'] = document_rep_feats
    all_documents_df['distractive_feats'] = document_distracting_feats

    data_for_evaluation = []
    for idx, row in all_documents_df.iterrows():
        item = {'documentID': row['documentID'],
               'fullText'  : row['fullText'],
               'descriptive_feats' :[x[1] for x in row['describtive_feats']],
               'distractive_feats' :[x[1] for x in row['distractive_feats']],
               'all_features'      :[x[1] for x in row['describtive_feats']] + [x[1] for x in row['distractive_feats']]}
        random.shuffle(item['all_features'])
        for idx, f in enumerate(item['all_features']):
            item['feature_{}'.format(idx)] = f

        data_for_evaluation.append(item)

    return data_for_evaluation

def print_eval_resutls(pandas_df):
    pandas_df['chatgpt-scorings-dict'] = pandas_df['chatgpt-scorings'].apply(lambda feats: {f[0].strip(): int(f[1]) for f in feats})
    pandas_df['descriptive_feats_scores'] = pandas_df.apply(lambda row: [row['chatgpt-scorings-dict'][f] if f in  row['chatgpt-scorings-dict'] else 0 for f in row['descriptive_feats']], axis=1)
    pandas_df['distracting_feats_scores'] = pandas_df.apply(lambda row: [row['chatgpt-scorings-dict'][f] if f in  row['chatgpt-scorings-dict'] else 0 for f in row['distractive_feats']], axis=1)
    pandas_df['descriptive_style_selected'] = pandas_df.apply(lambda row: np.mean(row['descriptive_feats_scores']) > np.mean(row['distracting_feats_scores']), axis=1)

    results_table = []
    describ_scores  = [s for scores in pandas_df['descriptive_feats_scores'].tolist() for s in scores]
    distract_scores = [s for scores in pandas_df['distracting_feats_scores'].tolist() for s in scores]
    
    print(tabulate.tabulate([
                                ['Descriptive feats', np.mean(describ_scores), np.median(describ_scores)],
                                ['Distracting feats', np.mean(distract_scores), np.median(distract_scores)]
                            ], headers=['Mean', 'Median']))
    print(pandas_df['descriptive_style_selected'].tolist().count(True)/len(pandas_df))

In [7]:
tfidf_data_for_evaluation = prepare_data_for_evaluation1(input_path, interp_space_path, interp_space_rep_path, feat_clm='llm_tfidf_rep', top_c=1, top_k=5)
cont_data_for_evaluation = prepare_data_for_evaluation1(input_path, interp_space_path, interp_space_rep_path, feat_clm='llm_con_rep', top_c=1, top_k=5)

# clusters: 147
# clusters: 147
# clusters: 147
# clusters: 147


In [8]:
tfidf_data_for_evaluation[0]

{'documentID': '1d50fcc5-ea3e-5e0e-921b-950dbd1f4834',
 'fullText': "For the electronics sector, the development of optical fibers has been transformative. Due to their ability to transmit data over long distances more quickly and reliably, they have grown to be an essential part of contemporary communication systems. We will look at several ways optical fibers have helped the electronics industry in this article.\n\n  What exactly are optical fibers?\n\n  Long-distance transmission of light signals is done using optical fibers, which are thin, flexible strands of glass or plastic. An optical fiber's core is made of a superior, ultra-pure glass substance that can transmit light with little loss. Total internal reflection is the process by which the light travels through the fiber by repeatedly hitting the walls of the core.\n\n  Benefits of optical fibers.\n\n  Optic fibers have a number of benefits over conventional copper wires. First off, they have much greater speed and range than 

In [151]:
#tfidf_results = style_generator.evaluate_style_features_in_text(data_for_evaluation[:20], 'evaluate_llm_tfidf_rep')
con_results = style_generator.evaluate_style_features_in_text(cont_data_for_evaluation[:20], 'evaluate_llm_cont_rep')

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: .datadreamer/evaluate_style_features_in_text/openai:gpt-3.5-turbo/evaluate_llm_cont_rep
[ 🤖 DataDreamer 💤 ] Step 'experiment-1.2-dataset' was previously run and saved, but was outdated. 😞
[ 🤖 DataDreamer 💤 ] Step 'experiment-1.2-dataset' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'experiment-1.2-dataset' finished and is saved to disk. 🎉
/home/ma4608/style_generation_pipeline/src-ipynb/../../style_generation_pipeline/styles/__init__.py:584: UserWarning: You did not specify `total_num_rows`, so we cannot automatically update the progress % for this step. Either specify map(..., total_num_rows=#) or, to disable this warning, specify map(.., auto_progress = False)
  ds = ds.map(lambda row: {
[ 🤖 DataDreamer 💤 ] Step 'experiment-1.2-dataset (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'experiment-1.2-dataset (map)' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'rate applicability of features in texts' was previously run and saved, but was ou

In [8]:
tfidf_results[['fullText', 'descriptive_feats', 'distractive_feats']].head(n=10)

NameError: name 'tfidf_results' is not defined

In [149]:
tfidf_results[['all_features', 'prompts', 'generations', 'chatgpt-scorings']].head()

all_features  \
0  [transitional elements are used, the author maintains a neutral tone in their work, the author maintains a professional tone in their work, the author skillfully incorporates transitional elements for smooth idea connections, The writer uses sentence length variation for visual impact., repetition is used, The writer uses varied sentence lengths for a smooth flow., the author uses a formal tone, the author uses simple sentence structures, the author uses various sentence structures in their work]   
1  [the author maintains a neutral tone in their work, The writer uses sentence length variation for visual impact., the author uses simple sentence structures, the author skillfully incorporates transitional elements for smooth idea connections, the author uses a formal tone, the author maintains a professional tone in their work, the author uses various sentence structures in their work, transitional elements are used, repetition is used, The writer uses varied sentence lengths for a smooth flow.]   
2  [the author uses various sentence structures in their work, the author skillfully incorporates transitional elements for smooth idea connections, the author maintains a professional tone in their work, The writer uses sentence length variation for visual impact., The writer uses varied sentence lengths for a smooth flow., repetition is used, the author uses simple sentence structures, the author maintains a neutral tone in their work, transitional elements are used, the author uses a formal tone]   
3  [The writer uses varied sentence lengths for a smooth flow., the author uses various sentence structures in their work, the author skillfully incorporates transitional elements for smooth idea connections, The writer uses sentence length variation for visual impact., the author uses simple sentence structures, transitional elements are used, the author maintains a professional tone in their work, repetition is used, the author maintains a neutral tone in their work, the author uses a formal tone]   
4                                                        [The writer uses different methods to create complex sentences., the author uses a formal tone, transitional elements are used, the author maintains a neutral tone in their work, the author uses various sentence structures in their work, the author maintains a professional tone in their work, sentence lengths are varied, the author's language style is varried, the author uses various language styles, the author uses simple sentence structures]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [147]:
print_eval_resutls(tfidf_results)

                     Mean    Median
-----------------  ------  --------
Descriptive feats    2.55         3
Distracting feats    2.56         3
0.3


In [117]:
print_eval_resutls(tfidf_results)

                     Mean    Median
-----------------  ------  --------
Descriptive feats    2.65         3
Distracting feats    2.82         3
0.05


In [152]:
print_eval_resutls(con_results)

                     Mean    Median
-----------------  ------  --------
Descriptive feats    2.1          3
Distracting feats    2.16         3
0.25


In [118]:
print_eval_resutls(con_results)

                     Mean    Median
-----------------  ------  --------
Descriptive feats    2.75         3
Distracting feats    2.68         3
0.5
